<a href="https://colab.research.google.com/github/judebebo32/ML_CLG/blob/main/Cedluloselignindata2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models


In [ ]:
data = pd.read_excel('/content/dataset_updated_copy.xlsx')
x = data.iloc[:, 0:8].values
y = data.iloc[:, 8:11].values

In [ ]:
# Define generator model
def make_generator_model():
    model = models.Sequential()
    model.add(layers.Dense(16, input_shape=(8,), activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(3))
    return model

# Define discriminator model
def make_discriminator_model():
    model = models.Sequential()
    model.add(layers.Dense(32, input_shape=(3,), activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Define discriminator loss function
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Define generator loss function
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
# Define training loop
@tf.function
def train_step(x_real, y_real):
    noise = tf.random.normal([len(x_real), 8])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake data using the generator
        x_fake = generator(noise, training=True)

        # Get discriminator predictions for real and fake data
        y_real_pred = discriminator(y_real, training=True)
        y_fake_pred = discriminator(x_fake, training=True)

        # Calculate losses
        gen_loss = generator_loss(y_fake_pred)
        disc_loss = discriminator_loss(y_real_pred, y_fake_pred)

    # Calculate gradients and apply to optimizer
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 0 - Generator Loss: 0.6905, Discriminator Loss: 0.6983
Epoch 50 - Generator Loss: 0.7121, Discriminator Loss: 0.6749
Epoch 100 - Generator Loss: 0.7457, Discriminator Loss: 0.6434
Epoch 150 - Generator Loss: 0.7575, Discriminator Loss: 0.6329
Epoch 200 - Generator Loss: 0.7904, Discriminator Loss: 0.6048
Epoch 250 - Generator Loss: 0.8200, Discriminator Loss: 0.5816
Epoch 300 - Generator Loss: 0.8697, Discriminator Loss: 0.5455
Epoch 350 - Generator Loss: 0.9019, Discriminator Loss: 0.5229
Epoch 400 - Generator Loss: 1.0267, Discriminator Loss: 0.4467
Epoch 450 - Generator Loss: 1.1123, Discriminator Loss: 0.4044
Epoch 500 - Generator Loss: 1.2039, Discriminator Loss: 0.3598
Epoch 550 - Generator Loss: 1.3898, Discriminator Loss: 0.2892
Epoch 600 - Generator Loss: 1.5043, Discriminator Loss: 0.2534
Epoch 650 - Generator Loss: 1.6604, Discriminator Loss: 0.2121
Epoch 700 - Generator Loss: 1.6570, Discriminator Loss: 0.2122
Epoch 750 - Generator Loss: 1.8307, Discriminator Loss: 0.

In [ ]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

    # Generate synthetic data using the trained generator model
    num_samples = len(x_val)
    noise = tf.random.normal([num_samples, 8])
    synthetic_data = generator(noise).numpy()

    # Convert the synthetic data to a pandas DataFrame
    synthetic_df = pd.DataFrame(np.hstack((x_val, synthetic_data)), columns=['Input 1', 'Input 2', 'Input 3', 'Input 4', 'Input 5', 'Input 6', 'Input 7', 'Input 8', 'Output 1', 'Output 2', 'Output 3'])




Epoch 0 - Generator Loss: 0.9673, Discriminator Loss: 2.7716
Epoch 50 - Generator Loss: 0.7207, Discriminator Loss: 1.8538
Epoch 100 - Generator Loss: 1.3362, Discriminator Loss: 1.4060
Epoch 150 - Generator Loss: 1.4667, Discriminator Loss: 1.2979
Epoch 200 - Generator Loss: 1.8429, Discriminator Loss: 1.1689
Epoch 250 - Generator Loss: 1.6728, Discriminator Loss: 1.0931
Epoch 300 - Generator Loss: 0.2334, Discriminator Loss: 2.5464
Epoch 350 - Generator Loss: 1.2047, Discriminator Loss: 1.3082
Epoch 400 - Generator Loss: 1.3461, Discriminator Loss: 1.1231
Epoch 450 - Generator Loss: 0.8840, Discriminator Loss: 1.2433
Epoch 500 - Generator Loss: 1.2760, Discriminator Loss: 1.0576
Epoch 550 - Generator Loss: 0.9576, Discriminator Loss: 1.1473
Epoch 600 - Generator Loss: 0.9760, Discriminator Loss: 1.1216
Epoch 650 - Generator Loss: 1.0747, Discriminator Loss: 1.0491
Epoch 700 - Generator Loss: 1.1263, Discriminator Loss: 0.9839
Epoch 750 - Generator Loss: 1.4253, Discriminator Loss: 0.

In [ ]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
fold = 1

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

    # Generate synthetic data using the trained generator model
    num_samples = len(x_val)
    noise = tf.random.normal([num_samples, 8])
    synthetic_data = generator(noise).numpy()

    # Convert the synthetic data to a pandas DataFrame
    synthetic_df = pd.DataFrame(np.hstack((x_val, synthetic_data)), columns=['Input 1', 'Input 2', 'Input 3', 'Input 4', 'Input 5', 'Input 6', 'Input 7', 'Input 8', 'Output 1', 'Output 2', 'Output 3'])

   # Export the synthetic data to an Excel file
    synthetic_df.to_excel(f'/content/sample_data/synthetic_data_fold{fold}.xlsx', index=False)

    fold += 1

Epoch 0 - Generator Loss: 1.1006, Discriminator Loss: 2.1350
Epoch 50 - Generator Loss: 0.8781, Discriminator Loss: 1.7173
Epoch 100 - Generator Loss: 0.6454, Discriminator Loss: 1.7402
Epoch 150 - Generator Loss: 0.6762, Discriminator Loss: 1.6369
Epoch 200 - Generator Loss: 0.7407, Discriminator Loss: 1.4842
Epoch 250 - Generator Loss: 0.7448, Discriminator Loss: 1.4008
Epoch 300 - Generator Loss: 0.9828, Discriminator Loss: 1.1777
Epoch 350 - Generator Loss: 0.8593, Discriminator Loss: 1.2691
Epoch 400 - Generator Loss: 0.6977, Discriminator Loss: 1.3403
Epoch 450 - Generator Loss: 0.7297, Discriminator Loss: 1.2576
Epoch 500 - Generator Loss: 0.6156, Discriminator Loss: 1.3295
Epoch 550 - Generator Loss: 0.6047, Discriminator Loss: 1.3381
Epoch 600 - Generator Loss: 0.6244, Discriminator Loss: 1.3080
Epoch 650 - Generator Loss: 0.7221, Discriminator Loss: 1.2222
Epoch 700 - Generator Loss: 0.7419, Discriminator Loss: 1.2253
Epoch 750 - Generator Loss: 0.5634, Discriminator Loss: 1.

In [ ]:
import matplotlib.pyplot as plt

# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
epoch_num = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        if epoch % 50 == 0:
            print(f'Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

        epoch_num.append(epoch)



Epoch 0 - Generator Loss: 1.1192, Discriminator Loss: 2.0514
Epoch 50 - Generator Loss: 1.3710, Discriminator Loss: 1.3119
Epoch 100 - Generator Loss: 1.3009, Discriminator Loss: 1.0470
Epoch 150 - Generator Loss: 1.6884, Discriminator Loss: 0.9031
Epoch 200 - Generator Loss: 0.4362, Discriminator Loss: 1.5715
Epoch 250 - Generator Loss: 0.5979, Discriminator Loss: 1.2944
Epoch 300 - Generator Loss: 1.7301, Discriminator Loss: 0.6153
Epoch 350 - Generator Loss: 1.4872, Discriminator Loss: 0.6991
Epoch 400 - Generator Loss: 1.1416, Discriminator Loss: 0.7950
Epoch 450 - Generator Loss: 0.4996, Discriminator Loss: 1.3081
Epoch 500 - Generator Loss: 1.1334, Discriminator Loss: 0.7390
Epoch 550 - Generator Loss: 1.2300, Discriminator Loss: 0.7043
Epoch 600 - Generator Loss: 1.1009, Discriminator Loss: 0.7465
Epoch 650 - Generator Loss: 0.3997, Discriminator Loss: 1.4836
Epoch 700 - Generator Loss: 1.1021, Discriminator Loss: 0.7521
Epoch 750 - Generator Loss: 0.6065, Discriminator Loss: 1.

In [ ]:
import pandas as pd

# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
mae = pd.DataFrame(columns=['Fold', 'Output 1', 'Output 2', 'Output 3'])

for fold, (train_index, val_index) in enumerate(kf.split(x), 1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    mae_fold = {'Fold': fold}
    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)

        # Calculate MAE for each output variable
        synthetic_data = generator(tf.random.normal([len(x_val), 8])).numpy()
        for i in range(3):
            mae_fold[f'Output {i+1}'] = tf.keras.metrics.mean_absolute_error(y_val[:, i], synthetic_data[:, i]).numpy()

        if epoch % 50 == 0:
            print(f'Fold {fold}, Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

        mae = mae.append(mae_fold, ignore_index=True)

# Export MAE data to Excel file
mae.to_excel('/content/sample_data/mae_data.xlsx', index=False)

Fold 1, Epoch 0 - Generator Loss: 0.8174, Discriminator Loss: 2.0731


<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 50 - Generator Loss: 0.1971, Discriminator Loss: 2.4500


<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 100 - Generator Loss: 0.4617, Discriminator Loss: 1.6813


...

<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 5, Epoch 900 - Generator Loss: 0.8626, Discriminator Loss: 1.0947


<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 5, Epoch 950 - Generator Loss: 0.8921, Discriminator Loss: 1.2019


<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-11-eb01c3c46eda>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

# New Section

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# Load data from Excel file
data = pd.read_excel('/content/New_data_2 - Copy.xlsx')

# Split input and output variables
x = data.iloc[:, :8].values
y = data.iloc[:, 8:].values

# Define generator and discriminator models
def make_generator_model():
    model = keras.Sequential([
        keras.layers.Dense(16, input_shape=(8,), activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(3, activation='linear')
    ])
    return model

def make_discriminator_model():
    model = keras.Sequential([
        keras.layers.Dense(16, input_shape=(3,), activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define GAN model
def make_gan_model(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential([
        generator,
        discriminator
    ])
    return model

# Define loss functions
binary_crossentropy = keras.losses.BinaryCrossentropy(from_logits=False)
mse = keras.losses.MeanSquaredError()

# Define optimizers
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Define training step for GAN model
@tf.function
def train_step(x_real, y_real):
    noise = tf.random.normal([len(x_real), 8])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        x_fake = generator(noise)
        y_fake = discriminator(x_fake)
        y_real_pred = discriminator(y_real)

        gen_loss = mse(y_real, x_fake)
        disc_loss_fake = binary_crossentropy(tf.zeros_like(y_fake), y_fake)
        disc_loss_real = binary_crossentropy(tf.ones_like(y_real_pred), y_real_pred)
        disc_loss = (disc_loss_fake + disc_loss_real) / 2

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss


# New Section

In [ ]:
!ls /content/sample_data/

anscombe.json		      synthetic_data_fold1.xlsx
california_housing_test.csv   synthetic_data_fold2.xlsx
california_housing_train.csv  synthetic_data_fold3.xlsx
mnist_test.csv		      synthetic_data_fold4.xlsx
mnist_train_small.csv	      synthetic_data_fold5.xlsx
README.md


In [ ]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
mae = pd.DataFrame(columns=['Fold', 'Output 1', 'Output 2', 'Output 3'])

for fold, (train_index, val_index) in enumerate(kf.split(x), 1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    generator = make_generator_model()
    discriminator = make_discriminator_model()
    gan = make_gan_model(generator, discriminator)

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        # Calculate MAE for each output variable
        synthetic_data = generator(tf.random.normal([len(x_val), 8])).numpy()
        mae_fold = {'Fold': fold}
        for i in range(3):
            mae_fold[f'Output {i+1}'] = tf.keras.metrics.mean_absolute_error(y_val[:, i], synthetic_data[:, i]).numpy()
        mae = mae.append(mae_fold, ignore_index=True)

        if epoch % 50 == 0:
            print(f'Fold {fold}, Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

    # Generate synthetic data and export to Excel file
    synthetic_data = generator(tf.random.normal([len(x), 8])).numpy()
    synthetic_df = pd.DataFrame(np.concatenate([x, synthetic_data], axis=1), columns=data.columns)
    synthetic_df.to_excel(f'/content/sample_data_fold{fold}.xlsx', index=False)

# Export MAE data to Excel file
mae.to_excel('/content/sample_data.xlsx', index=False)

<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 0 - Generator Loss: 645.0358, Discriminator Loss: 0.3554


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 50 - Generator Loss: 99.3816, Discriminator Loss: 2.2384


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

...

Fold 2, Epoch 900 - Generator Loss: 54.6975, Discriminator Loss: 3.4066


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 2, Epoch 950 - Generator Loss: 55.5560, Discriminator Loss: 3.4188


<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-30-e24a2dc6521e>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

KeyError: ignored

In [ ]:
# Define optimizers
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Define training step for GAN model
@tf.function
def train_step(x_real, y_real):
    noise = tf.random.normal([len(x_real), 8])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        x_fake = generator(noise)
        y_fake = discriminator(x_fake)
        y_real_pred = discriminator(y_real)

        gen_loss = mse(y_real, x_fake)
        disc_loss_fake = binary_crossentropy(tf.zeros_like(y_fake), y_fake)
        disc_loss_real = binary_crossentropy(tf.ones_like(y_real_pred), y_real_pred)
        disc_loss = (disc_loss_fake + disc_loss_real) / 2

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.build(generator.trainable_variables)
    discriminator_optimizer.build(discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
# Run GAN training loop for n epochs
n = 1000
batch_size = 32
kf = KFold(n_splits=5)
gen_losses = []
disc_losses = []
mae = pd.DataFrame(columns=['Fold', 'Output 1', 'Output 2', 'Output 3'])

for fold, (train_index, val_index) in enumerate(kf.split(x), 1):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    generator = make_generator_model()
    discriminator = make_discriminator_model()
    gan = make_gan_model(generator, discriminator)

    for epoch in range(n):
        for i in range(0, len(x_train), batch_size):
            x_batch_real = x_train[i:i + batch_size]
            y_batch_real = y_train[i:i + batch_size]
            gen_loss, disc_loss = train_step(x_batch_real, y_batch_real)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)

        # Calculate MAE for each output variable
        synthetic_data = generator(tf.random.normal([len(x_val), 8])).numpy()
        mae_fold = {'Fold': fold}
        for i in range(3):
            mae_fold[f'Output {i+1}'] = tf.keras.metrics.mean_absolute_error(y_val[:, i], synthetic_data[:, i]).numpy()
        mae = mae.append(mae_fold, ignore_index=True)

        if epoch % 50 == 0:
            print(f'Fold {fold}, Epoch {epoch} - Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 0 - Generator Loss: 639.7891, Discriminator Loss: 0.3483


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 1, Epoch 50 - Generator Loss: 97.2751, Discriminator Loss: 2.5994


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

...

Fold 2, Epoch 900 - Generator Loss: 54.8442, Discriminator Loss: 1.8182


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

Fold 2, Epoch 950 - Generator Loss: 55.9813, Discriminator Loss: 1.8298


<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mae = mae.append(mae_fold, ignore_index=True)
<ipython-input-32-aeb94257c4c4>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

KeyError: ignored